In [190]:
import pandas as pd
import re

In [191]:
#import files as dataframes for easy manipulation and cleaning
kl=pd.read_csv("mudah_kl_add.csv")
sel=pd.read_csv("mudah_sel_add.csv")

In [192]:
#compare columns of both files
kl.columns==sel.columns

array([ True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True])

In [193]:
#merge into one dataframe
#after appending the data, the index needs to be changed as you will see two observations with index 0 for example
klsel=pd.concat([kl,sel]).reset_index(drop=True)

In [194]:
print(kl.shape)
print(sel.shape)
print(klsel.shape)

(20507, 13)
(20597, 13)
(41104, 13)


In [195]:
#inspect the data
klsel.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 41104 entries, 0 to 41103
Data columns (total 13 columns):
Unnamed: 0      41104 non-null int64
Unnamed: 0.1    41104 non-null int64
Area            41104 non-null object
Price           41104 non-null object
Bedroom         38414 non-null object
Bathroom        38421 non-null object
Type            38440 non-null object
SquareFeet      39383 non-null object
link            41104 non-null object
Title           41104 non-null object
Oth_Info        41104 non-null object
Carpark         16316 non-null float64
Facilities      25856 non-null object
dtypes: float64(1), int64(2), object(10)
memory usage: 4.1+ MB


In [196]:
klsel.head()

,Unnamed: 0,Unnamed: 0.1,Area,Price,Bedroom,Bathroom,Type,SquareFeet,link,Title,Oth_Info,Carpark,Facilities
0,0,0,OUG,RM 3 999 000,8 Bedrooms,8 Bathrooms,Houses,7000 sq.ft,https://www.mudah.my/Happy+Garden+Bungalow+3+S...,Freehold,Non Bumi Lot,NaN,NaN
1,1,1,Desa ParkCity,RM 3 300 000,4 Bedrooms,3 Bathrooms,Houses,2379 sq.ft,https://www.mudah.my/Desa+park+city+zenia+end+...,Freehold,Non Bumi Lot,NaN,"Swimming Pool, Gymnasium, Playground, Jogging ..."
2,2,2,Wangsa Maju,RM 590 000,4 Bedrooms,2 Bathrooms,Apartments,1065 sq.ft,https://www.mudah.my/QN+Residence+Jalan+Wangsa...,Leasehold,Non Bumi Lot,2.0,"Swimming Pool, Gymnasium, Playground, Jogging ..."
3,3,2,Wangsa Maju,RM 590 000,4 Bedrooms,2 Bathrooms,Apartments,1065 sq.ft,https://www.mudah.my/QN+Residence+Jalan+Wangsa...,Leasehold,Non Bumi Lot,2.0,"Swimming Pool, Gymnasium, Playground, Jogging ..."
4,4,4,Ampang Hilir,RM 460 000,2 Bedrooms,2 Bathrooms,Apartments,743 sq.ft,https://www.mudah.my/PANDAN+PERDANA+new+condo+...,Leasehold,Non Bumi Lot,1.0,"Swimming Pool, Gymnasium, Tennis Court, Squash..."


In [197]:
#delete unused columns
klsel=klsel.drop(['Unnamed: 0', 'Unnamed: 0.1','link'], axis=1)
klsel.columns

Index(['Area', 'Price', 'Bedroom', 'Bathroom', 'Type', 'SquareFeet', 'Title',
       'Oth_Info', 'Carpark', 'Facilities'],
      dtype='object')

In [198]:
klsel.Area.fillna("NA",inplace=True)
klsel.Type.fillna("NA",inplace=True)
klsel.Title.fillna("NA",inplace=True)
klsel.Oth_Info.fillna("NA",inplace=True)
klsel.Facilities.fillna("NA",inplace=True)

In [199]:
#change type of Area, Type, Title, Oth_Info and Facilities into String as they are categorical variables
klsel['Area']=klsel.Area.astype('str')
klsel['Type']=klsel.Type.astype('str')
klsel['Title']=klsel.Title.astype('str')
klsel['Oth_Info']=klsel.Oth_Info.astype('str')
klsel['Facilities']=klsel.Facilities.astype('str')

In [200]:
#Assess the unique values
#klsel["Area"].unique() #No NA but with blank spaces in the string
#klsel["Type"].unique() #Missing value replaced with NA successfully
#klsel['Title'].unique() #'-' is found, have to replace with NA
#klsel['Oth_Info'].unique() #'-' is found, have to replace with NA
#klsel['Facilities'].unique() #Missing value replaced with NA successfully

In [201]:
klsel.Price.fillna("NaN",inplace=True)
klsel.Bedroom.fillna("NaN",inplace=True)
klsel.Bathroom.fillna("NaN",inplace=True)
klsel.Carpark.fillna("NaN",inplace=True)
klsel.SquareFeet.fillna("NaN",inplace=True)

In [202]:
#strip blank spaces in all columns (left and right of the strings)
klsel= klsel.apply(lambda x: x.str.strip() if x.dtype == "object" else x)

In [203]:
#strip blank spaces within the strings for price and strip RM in each row
#for eg. RM 3 999 000 with regular expression
klsel['Price']=klsel.Price.apply(lambda y: y.replace(" ","").replace("RM",""))

In [204]:
#strip units from columns
klsel['Bedroom']=klsel.Bedroom.astype('str')
klsel['Bathroom']=klsel.Bathroom.astype('str')
klsel['Bedroom']=klsel.Bedroom.apply(lambda z: z.replace("Bedrooms","").replace("Bedroom",""))
klsel['Bathroom']=klsel.Bathroom.apply(lambda a: a.replace("Bathrooms","").replace("Bathroom",""))

In [205]:
#replace '-' with NA
klsel['Title']=klsel.Title.apply(lambda z:z.replace("-","NA"))
klsel['Oth_Info']=klsel.Oth_Info.apply(lambda z:z.replace("-","NA"))

Convert acres into square feet<br>
1.Create a dummy column to store the unit of measurement<br>
2.select by condition only rows with acres<br>
3.convert the selected rows from acres to square feet with a factor of 43560

In [206]:
#Create a dummy column to store the unit of measurement
klsel['UOM'] = klsel.SquareFeet.astype(str).apply(lambda x : 'Acres' if 'Acres' in x else 'SqFt')

In [207]:
#strip all UOM from the column SquareFeet
klsel['SquareFeet']=klsel.SquareFeet.astype(str).apply(lambda y:y.replace("Acres","").replace("sq.ft",""))

In [208]:
#Select only rows with UOM==acres and multiply the value in that row with 43560
klsel['SquareFeet']=klsel.SquareFeet.astype('float')
klsel.loc[klsel.UOM=="Acres", 'SquareFeet'] = klsel.SquareFeet*43560

In [209]:
#Delete dummy column UOM
klsel=klsel.drop(['UOM'], axis=1)

In [210]:
#change type of Price,Bedroom, Bathroom, Carpark and SquareFeet into int or float as they are the numerical variables
#before that, it is treated as string for easy manipulation
klsel['Price']=klsel.Price.astype('float')
klsel['Bedroom']=klsel.Bedroom.astype('float')
klsel['Bathroom']=klsel.Bathroom.astype('float')
klsel['Carpark']=klsel.Carpark.astype('float')

In [212]:
#write the final output into a single file
klsel.to_csv('cleaned_houseprice.csv', index=False)